In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_json("hf://datasets/Moaaz55/skin_cancer_questions_answers/dataset.json", lines=True)

,Question,Answer,Input
0,What is skin cancer?,Skin cancer is a malignant neoplasia that orig...,
1,What are the main types of skin cancer?,The main types are melanoma and non-melanoma s...,
2,What is the most common type of skin cancer?,Basal cell carcinoma (BCC) is the most common ...,
3,What is the most dangerous type of skin cancer?,Malignant melanoma (MM) is the most dangerous ...,
4,What is a cutaneous carcinoma?,Cutaneous carcinomas are malignant neoplasias ...,
...,...,...,...
793,"What does the term ""perineural invasion"" refer...",The spread of cancer cells around nerves,
794,What is a major risk associated with skin canc...,Increased risk of squamous cell carcinoma,
795,What does MTX refer to?,Methotrexate,
796,What is a typical use of acitretin in transpla...,It is used for those at higher risk of skin ca...,


In [ ]:
#CONVERTIR LAS RESPUESTAS A AUDIO CON LA LIBRERIA TTS.

from gtts import gTTS
from playsound import playsound

# AQUI ESPECIFICAMOS PARAMETROS COMO EL DIRECTORIO DE SALIDA, EL LENGUAJE Y LA COLUMNA DE LAS RESPUESTAS.

OUT_DIR = "output_audio"
LANG = "es"
TEXT_COL = "answer"

# A CONTINIUACIÓN HARE EL TRATAMIENTO DE LAS ANSWERS.

# AQUI LO QUE HACEMOS ES CREAR LOS DIRECTORIOS DE AUDIO Y METADATA.
def directoris(root):
    audio_dir = os.path.join(root, "audio")
    metadata_dir = os.path.join(root, "metadata")
    os.makedirs(audio_dir, exist_ok=True)
    os.makedirs(metadata_dir, exist_ok=True)
    return audio_dir, metadata_dir

# ESTA FUNCION SIRVE PARA EVITAR DUPLICADOS. SI DETECTA UN DUPLICAOD EN ANSWERR, DEVUELVE LA MISMA RESPUESTA QUE YA TENIA.
def md5(s: str) -> str:
    return hashlib.md5(s.encode("utf-8")).hexdigest()


def txt_to_bytes(text: str, lang: str) -> bytes:
    buf = io.BytesIO()
    gTTS(text=text, lang=lang).write_to_fp(buf)
    buf.seek(0)
    return buf.read()


def mp3_duration(b: bytes):
    try:
        return float(MP3(io.BytesIO(b)).info.length)
    except Exception:
        return None
    
def answers_to_audio(df: pd.DataFrame):
    audio_dir, meta_dir = ensure_dirs(OUT_DIR)
    df_out = df.copy()

    df_out["Answer_audio_relpath"] = None
    df_out["Answer_duration_sec"]  = None
    df_out["Answer_size_bytes"]    = None
    df_out["Answer_text_md5"]      = None

    total = len(df_out)
    for i, text in enumerate(df_out[TEXT_COL].astype(str)):
        if not text.strip():
            continue

        h = md5(text)
        filename = f"answer_{h}.mp3"
        abspath  = os.path.join(audio_dir, filename)
        relpath  = os.path.join("audio", filename)

        # solo generar si no existe
        if not os.path.exists(abspath):
            mp3 = tts_bytes(text, LANG)
            with open(abspath, "wb") as f:
                f.write(mp3)
            size = len(mp3)
            dur  = mp3_duration(mp3)
        else:
            with open(abspath, "rb") as f:
                data = f.read()
            size = len(data)
            dur  = mp3_duration(data)

        df_out.at[i, "Answer_audio_relpath"] = relpath.replace("\\", "/")
        df_out.at[i, "Answer_duration_sec"]  = dur
        df_out.at[i, "Answer_size_bytes"]    = size
        df_out.at[i, "Answer_text_md5"]      = h

        if (i+1) % 50 == 0 or i+1 == total:
            print(f"[{i+1}/{total}] {relpath} ({dur:.2f}s)")

    # Guardar dataset enriquecido
    today = datetime.utcnow().strftime("%Y-%m-%d")
    out_latest = os.path.join(meta_dir, "answers_dataset-latest.parquet")
    out_dated  = os.path.join(meta_dir, f"answers_dataset-{today}.parquet")

    df_out.to_parquet(out_latest, index=False)
    df_out.to_parquet(out_dated,  index=False)

    print("\n Listo")
    print("Audios →", os.path.abspath(audio_dir))
    print("Nuevo dataset →", out_latest)
    return df_out















In [14]:
from transformers import AutoModelForVision2Seq, AutoProcessor

model_id = "abaryan/DrDiag_qwen2vl_Ham10000"


In [ ]:

processor = Qwen2VLProcessor.from_pretrained(model_id)
model = Qwen2VLForConditionalGeneration.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

# Example usage
inputs = processor(images="lesion.jpg", text="Diagnose the lesion and provide bounding box.", return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(processor.decode(outputs[0], skip_special_tokens=True))

`torch_dtype` is deprecated! Use `dtype` instead!
